## [Day 1: Calorie Counting](https://adventofcode.com/2022/day/1)

In [2]:
elves = [0]
for line in eachline("1.txt")
    if line == ""
        push!(elves, 0)
        continue
    end
    elves[end] += parse(Int, line)
end
sort!(elves)
elves[end], sum(elves[end-2:end])

(70374, 204610)

## [Day 2: Rock Paper Scissors](https://adventofcode.com/2022/day/2)

In [3]:
games = split.(readlines("2.txt"))

score = Dict([
    ["A", "X"] => 3 + 1,
    ["B", "X"] => 0 + 1,
    ["C", "X"] => 6 + 1,
    ["A", "Y"] => 6 + 2,
    ["B", "Y"] => 3 + 2,
    ["C", "Y"] => 0 + 2,
    ["A", "Z"] => 0 + 3,
    ["B", "Z"] => 6 + 3,
    ["C", "Z"] => 3 + 3,
])
sum(score[g] for g in games)

10994

In [4]:
score = Dict([
    ["A", "X"] => 0 + 3,
    ["B", "X"] => 0 + 1,
    ["C", "X"] => 0 + 2,
    ["A", "Y"] => 3 + 1,
    ["B", "Y"] => 3 + 2,
    ["C", "Y"] => 3 + 3,
    ["A", "Z"] => 6 + 2,
    ["B", "Z"] => 6 + 3,
    ["C", "Z"] => 6 + 1,
])
sum(score[g] for g in games)

12526

## [Day 3: Rucksack Reorganization](https://adventofcode.com/2022/day/3)

In [5]:
data_3 = readlines("3.txt")

halves(x) = (x[1:div(length(x), 2)], x[div(length(x), 2)+1:end])
priority(c) = isuppercase(c) ? Int(c) - 64 + 26 : Int(c) - 96

sum(priority(only(intersect(Set.(halves(x))...))) for x in data_3),
sum(priority(only(intersect(Set.(x)...))) for x in Iterators.partition(data_3, 3))

(7863, 2488)

## [Day 4: Camp Cleanup](https://adventofcode.com/2022/day/4)

In [6]:
data_4 = readlines("4.txt")
ranges = [range(parse.(Int, split(x, '-'))...) for x in reduce(vcat, permutedims.(split.(data_4, ',')))]

sum(issubset(x, y) || issubset(y, x) for (x, y) in eachrow(ranges)),
sum(!isempty(intersect(x, y)) for (x, y) in eachrow(ranges))

(584, 933)

## [Day 5: Supply Stacks](https://adventofcode.com/2022/day/5)

In [7]:
data_5 = readlines("5.txt")
stacks = [filter(!=(' '), reverse(r)) for r in eachrow(hcat(collect.([l[2:4:end] for l in data_5[1:8]])...))]
instructions = data_5[11:end]

function cratemover9000(stacks, instructions)
    stacks = deepcopy(stacks)
    for i in instructions
        n, s, e = parse.(Int, match(r"move (\d+) from (\d+) to (\d+)", i).captures)
        for j in 1:n
            push!(stacks[e], pop!(stacks[s]))
        end
    end
    prod(last.(stacks))
end

function cratemover9001(stacks, instructions)
    stacks = deepcopy(stacks)
    for i in instructions
        n, s, e = parse.(Int, match(r"move (\d+) from (\d+) to (\d+)", i).captures)
        append!(stacks[e], stacks[s][end-n+1:end])
        for j in 1:n
            pop!(stacks[s])
        end
    end
    prod(last.(stacks))
end

cratemover9000(stacks, instructions), cratemover9001(stacks, instructions)

("VJSFHWGFT", "LCTQFBVZV")

## [Day 6: Tuning Trouble](https://adventofcode.com/2022/day/6)

In [8]:
data_6 = readline("6.txt")

end_of_marker(data, n) = findfirst(i -> allunique(data[i:i+n-1]), 1:length(data)) + n - 1

end_of_marker(data_6, 4), end_of_marker(data_6, 14)

(1175, 3217)

## [Day 7: No Space Left On Device](https://adventofcode.com/2022/day/7)

In [9]:
data_7 = readlines("7.txt")

function build_filesystem(terminal_output)
    filesystem = Dict()
    path = []
    for l in data_7
        if l == "\$ cd /"
            path = []
        elseif l == "\$ cd .."
            pop!(path)
        elseif startswith(l, "\$ cd")
            push!(path, last(split(l)))
        elseif l == "\$ ls"
            continue
        else
            cwd = foldl((d, x) -> d[x], path; init = filesystem)
            if startswith(l, "dir")
                get!(cwd, last(split(l)), Dict())
            else
                size, name = split(l)
                get!(cwd, name, parse(Int, size))
            end
        end
    end
    filesystem
end

function directory_sizes(filesystem)
    sizes = Int[]
    crawl(directory) = directory isa Dict ? last(push!(sizes, sum(crawl.(values(directory))))) : directory
    crawl(filesystem)
    sort!(sizes)
end

sizes = directory_sizes(build_filesystem(data_7))
sum(filter(<=(100000), sizes)), sizes[findfirst(>=(sizes[end] - 40000000), sizes)]

(1792222, 1112963)

## [Day 8: Treetop Tree House](https://adventofcode.com/2022/day/8)

In [10]:
data_8 = readlines("8.txt")
trees = parse.(Int, reduce(hcat, collect.(data_8)))

function forward_visible(trees)
    visible = falses(size(trees))
    h = -1
    for (i, t) in enumerate(trees)
        if t > h
            h = t
            visible[i] = true
        end
    end
    visible
end

sum(
    mapslices(forward_visible, trees; dims = 1) .|
    mapslices(forward_visible, trees; dims = 2) .|
    mapslices(reverse ∘ forward_visible ∘ reverse, trees; dims = 1) .|
    mapslices(reverse ∘ forward_visible ∘ reverse, trees; dims = 2),
)

1827

In [11]:
function forward_viewing_distance(trees)
    distance = zeros(Int, length(trees))
    for (i, t) in enumerate(trees)
        j = findfirst(>=(t), trees[i+1:end])
        distance[i] = isnothing(j) ? length(trees) - i : j
    end
    distance
end

maximum(
    mapslices(forward_viewing_distance, trees; dims = 1) .* mapslices(forward_viewing_distance, trees; dims = 2) .*
    mapslices(reverse ∘ forward_viewing_distance ∘ reverse, trees; dims = 1) .*
    mapslices(reverse ∘ forward_viewing_distance ∘ reverse, trees; dims = 2),
)

335580

## [Day 9: Rope Bridge](https://adventofcode.com/2022/day/9)

In [12]:
using StaticArrays

In [13]:
data_9 = readlines("9.txt")

function tail_locations(moves, num_knots = 2)
    directions = Dict("L" => SVector(-1, 0), "R" => SVector(1, 0), "U" => SVector(0, 1), "D" => SVector(0, -1))
    r = fill(SVector(0, 0), num_knots)
    ts = Set((r[end],))
    for (d, n) in split.(moves)
        for i in 1:parse(Int, n)
            r[1] += directions[d]
            for k in 2:length(r)
                if maximum(abs.(r[k-1] - r[k])) > 1
                    r[k] += clamp.(r[k-1] - r[k], -1, 1)
                end
            end
            push!(ts, r[end])
        end
    end
    ts
end

length(tail_locations(data_9, 2)), length(tail_locations(data_9, 10))

(6030, 2545)

## [Day 10: Cathode-Ray Tube](https://adventofcode.com/2022/day/10)

In [14]:
using SparseArrays

In [15]:
data_10 = readlines("10.txt")

function cathode_ray_tube(ops, interval = 40, offset = 20)
    incremements = [startswith(x, ('n', 'a')) ? 0 : parse(Int, x) for x in reduce(vcat, split.(data_10))]
    x = cumsum(prepend!(incremements, 1))
    @show sum(x[t] * t for t in 20:40:220)
    sparse(permutedims(-1:38 .<= reshape(x[1:end-1], 40, 6) .<= 1:40))
end

cathode_ray_tube(data_10)

sum((x[t] * t for t = 20:40:220)) = 13440


6×40 SparseMatrixCSC{Bool, Int64} with 106 stored entries:
⣏⡱⢸⠭⡂⢉⠝⢰⢉⡂⣏⡱⢰⣉⡆⢉⠝⢰⣉⡆
⠃⠀⠘⠒⠁⠓⠒⠈⠒⠃⠃⠑⠘⠀⠃⠓⠒⠘⠀⠃

## [Day 11: Monkey in the Middle](https://adventofcode.com/2022/day/11)

In [16]:
data_11 = readlines("11.txt")

struct Monkey{O,T}
    items::Vector{Int}
    operation::O
    divisor::Int
    target::T
end
function Monkey(def::Vector{String})
    items = parse.(Int, split(def[2][19:end], ','))
    operation = eval(Meta.parse("old -> " * def[3][20:end]))
    divisor, true_monkey, false_monkey = parse.(Int, (def[4][22:end], def[5][30:end], def[6][31:end]))
    Monkey(items, operation, divisor, x -> x % divisor == 0 ? true_monkey : false_monkey)
end

monkeys = [Monkey(data_11[i:i+5]) for i in 1:7:length(data_11)]

function monkey_business(monkeys, rounds = 20, div3 = true)
    divisor_lcm = lcm([monkey.divisor for monkey in monkeys])
    monkeys = deepcopy(monkeys)
    inspection_counts = zeros(Int, length(monkeys))
    for _ in 1:rounds
        for (i, monkey) in enumerate(monkeys)
            while !isempty(monkey.items)
                inspection_counts[i] += 1
                worry = monkey.operation(popfirst!(monkey.items))
                worry = div3 ? worry ÷ 3 : worry % divisor_lcm
                push!(monkeys[monkey.target(worry)+1].items, worry)
            end
        end
    end
    inspection_counts
end

prod(sort!(monkey_business(monkeys))[end-1:end]), prod(sort!(monkey_business(monkeys, 10000, false))[end-1:end])

(120056, 21816744824)

## [Day 12: Hill Climbing Algorithm](https://adventofcode.com/2022/day/12)

In [17]:
using DataStructures

In [18]:
data_12 = reduce(hcat, collect.(readlines("12.txt")))

function shortest_path(elevations, from_any_a = false)
    directions = (CartesianIndex(0, 1), CartesianIndex(0, -1), CartesianIndex(1, 0), CartesianIndex(-1, 0))
    s, e = findfirst(==('S'), elevations), findfirst(==('E'), elevations)
    elevations = deepcopy(elevations)
    elevations[[s, e]] .= 'a', 'z'
    pq = PriorityQueue(e => 0)
    while from_any_a ? elevations[first(peek(pq))] != 'a' : first(peek(pq)) != s
        x, c = dequeue_pair!(pq)
        for d in directions
            if x + d in CartesianIndices(elevations) && elevations[x+d] >= elevations[x] - 1
                pq[x+d] = min(get!(pq, x + d, c + 1), c + 1)
            end
        end
    end
    last(peek(pq))
end

shortest_path(data_12), shortest_path(data_12, true)

(383, 377)

## [Day 13: Distress Signal](https://adventofcode.com/2022/day/13)

In [19]:
data_13 = readlines("13.txt")

struct IntWrapper
    x::Int
end
Base.isless(a::IntWrapper, b::IntWrapper) = a.x < b.x
Base.isless(a::Vector, b::IntWrapper) = a < [b]
Base.isless(a::IntWrapper, b::Vector) = [a] < b
Base.isequal(a::Vector, b::IntWrapper) = isequal(a, [b])
Base.isequal(a::IntWrapper, b::Vector) = isequal([a], b)
wrap_ints(x::Int) = IntWrapper(x)
wrap_ints(x::Vector) = wrap_ints.(x)

packets = wrap_ints.(eval.(Meta.parse.(filter(!isempty, data_13))))
sorted_packets = sort!([packets; [[[IntWrapper(2)]]]; [[[IntWrapper(6)]]]])
sum(findall(packets[1:2:end] .< packets[2:2:end])),
findfirst(==([[IntWrapper(2)]]), sorted_packets) * findfirst(==([[IntWrapper(6)]]), sorted_packets)

(5340, 21276)

## [Day 14: Regolith Reservoir](https://adventofcode.com/2022/day/14)

In [20]:
using OffsetArrays

In [21]:
data_14 = readlines("14.txt")
paths = [[parse.(Int, split(xy, ',')) for xy in path] for path in split.(data_14, " -> ")]

function count_sand(paths, include_floor = false)
    directions = (CartesianIndex(0, 1), CartesianIndex(-1, 1), CartesianIndex(1, 1))
    points = reduce(hcat, reduce(vcat, paths))
    (x_min, x_max), (y_min, y_max) = extrema(points; dims = 2)
    if include_floor
        y_max += 2
        x_min -= y_max
        x_max += y_max
        push!(paths, [[x_min, y_max], [x_max, y_max]])
    end

    grid = zeros(Int, x_min:x_max, 0:y_max)
    for p in paths
        for i in 1:length(p)-1
            grid[(x -> range(x...)).(minmax.(p[i], p[i+1]))...] .= 1
        end
    end
    while !include_floor || grid[CartesianIndex(500, 0)] == 0
        sand = CartesianIndex(500, 0)
        while true
            for d in directions
                sand + d in CartesianIndices(grid) || return grid
                if grid[sand+d] == 0
                    sand += d
                    @goto keep_falling
                end
            end
            grid[sand] = 2
            break
            @label keep_falling
        end
    end
    grid
end

sum(count_sand(paths) .== 2), sum(count_sand(paths, true) .== 2)

(618, 26358)

## [Day 15: Beacon Exclusion Zone](https://adventofcode.com/2022/day/15)

In [22]:
using StaticArrays

In [23]:
data_15 = readlines("15.txt")
sensor_beacon_pairs = [
    (SVector(sx, sy), SVector(bx, by)) for (sx, sy, bx, by) in [
        parse.(Int, match(r"Sensor at x=(-?\d+), y=(-?\d+): closest beacon is at x=(-?\d+), y=(-?\d+)", l).captures) for l in data_15
    ]
]

function combine_ranges!(ranges)
    sort!(filter!(!isempty, ranges); by = first)
    i = 2
    while i <= length(ranges)
        if last(ranges[i-1]) >= first(ranges[i])
            ranges[i-1] = first(ranges[i-1]):max(last(ranges[i-1]), last(ranges[i]))
            deleteat!(ranges, i)
        else
            i += 1
        end
    end
    ranges
end

function no_beacon_ranges_by_y(sensor_beacon_pairs, y)
    combine_ranges!([(dx = sum(abs.(s - b)) - abs(s[2] - y); s[1]-dx:s[1]+dx) for (s, b) in sensor_beacon_pairs])
end

function find_beacon(sensor_beacon_pairs, x_range = 0:4000000, y_range = 0:4000000)
    for y in y_range
        no_beacon_ranges = no_beacon_ranges_by_y(sensor_beacon_pairs, y)
        if !any(issubset(x_range, r) for r in no_beacon_ranges)
            return SVector(last(no_beacon_ranges[findfirst(r -> last(r) in x_range, no_beacon_ranges)]) + 1, y)
        end
    end
end

sum(length.(no_beacon_ranges_by_y(sensor_beacon_pairs, 2000000))) -
length(unique(filter(x -> x[2] == 2000000, last.(sensor_beacon_pairs)))),
find_beacon(sensor_beacon_pairs)' * SVector(4000000, 1)

(4876693, 11645454855041)

## [Day 16: Proboscidea Volcanium](https://adventofcode.com/2022/day/16)

In [24]:
using OffsetArrays

In [25]:
data_16 = readlines("16.txt")

function optimal_pressure(scan)
    flow_rates_dict = Dict(v[7:8] => parse(Int, match(r"\d+", v).match) for v in scan)
    out_edges_dict = Dict(v[7:8] => split(match(r"valves? (.*)", v).captures[1], ", ") for v in scan)
    sorted_flow_rates = sort(collect(flow_rates_dict), by = last, rev = true)
    location_index = Dict(Pair.(first.(sorted_flow_rates), 1:length(sorted_flow_rates)))
    out_edges_by_index = [[location_index[x] for x in out_edges_dict[k]] for k in first.(sorted_flow_rates)]

    num_nonzero_valves = findlast(kv -> last(kv) > 0, sorted_flow_rates)
    valve_states = 0:2^num_nonzero_valves-1
    valve_state_release_rate = [
        sum(
            v * (valve_state & (1 << x) > 0) for
            (x, v) in zip(0:num_nonzero_valves-1, last.(sorted_flow_rates[1:num_nonzero_valves]))
        ) for valve_state in valve_states
    ]

    optimal_release_to_come = zeros(Int, 0:30, 1:length(location_index), valve_states) .- 1
    optimal_release_to_come[0, location_index["AA"], 0] = 0

    for t in 1:30
        for location in 1:length(location_index)
            for valve_state in valve_states
                release_to_come = optimal_release_to_come[t-1, location, valve_state]
                if release_to_come >= 0
                    current_release = valve_state_release_rate[valve_state+1]
                    if location <= num_nonzero_valves && (valve_state & (1 << (location - 1)) == 0)
                        new_valve_state = valve_state | (1 << (location - 1))
                        optimal_release_to_come[t, location, new_valve_state] = max(
                            optimal_release_to_come[t, location, new_valve_state],
                            release_to_come + current_release,
                        )
                    end
                    for next_location in out_edges_by_index[location]
                        optimal_release_to_come[t, next_location, valve_state] = max(
                            optimal_release_to_come[t, next_location, valve_state],
                            release_to_come + current_release,
                        )
                    end
                end
            end
        end
    end

    optimal_release_to_come_to_any_location = maximum(optimal_release_to_come; dims = 2)
    maximum(optimal_release_to_come[30, :, :]),
    maximum(
        optimal_release_to_come_to_any_location[26, 1, my_valve_state] +
        optimal_release_to_come_to_any_location[26, 1, elephant_valve_state] for my_valve_state in valve_states for
        elephant_valve_state in valve_states if my_valve_state & elephant_valve_state == 0
    )
end

optimal_pressure(data_16)

(2320, 2967)

## [Day 17: Pyroclastic Flow](https://adventofcode.com/2022/day/17)

In [26]:
data_17 = readline("17.txt")

function falling_rocks(moves, num_rocks = 2022)
    rocks = [ones(Int, 1, 4), [0 1 0; 1 1 1; 0 1 0], [1 1 1; 0 0 1; 0 0 1], ones(Int, 4, 1), ones(Int, 2, 2)]

    function fall_until_full_row((initial_move_index, initial_rock_index, tower_start), max_num_rocks)
        max_num_rocks == 0 && return (false, (initial_move_index, initial_rock_index, tower_start), 0, 0)
        tower = zeros(Int, 1000, 7)
        tower[CartesianIndices(tower_start)] .= tower_start

        m = r = 0
        rock_level = size(tower_start, 1)
        offset = CartesianIndex(rock_level + 3, 2)
        while true
            m += 1
            rock = rocks[mod(initial_rock_index + r, 1:length(rocks))]
            for offset_delta in (
                CartesianIndex(0, moves[mod(initial_move_index + m - 1, 1:length(moves))] == '>' ? 1 : -1),
                CartesianIndex(-1, 0),
            )
                new_offset = offset + offset_delta
                if all(
                    rock[i] == 0 || (i + new_offset in CartesianIndices(tower) && tower[i+new_offset] == 0) for
                    i in CartesianIndices(rock)
                )
                    offset = new_offset
                elseif offset_delta == CartesianIndex(-1, 0)
                    r += 1
                    tower[offset.+CartesianIndices(rock)] .+= rock
                    rock_level = max(rock_level, size(rock, 1) + offset.I[1])

                    full_row_index =
                        findlast(all(==(1), row) for row in eachrow(tower[offset.I[1].+(1:size(rock, 1)), :]))
                    mi, ri = mod(initial_move_index + m, 1:length(moves)), mod(initial_rock_index + r, 1:length(rocks))
                    if !isnothing(full_row_index)
                        num_rows_removed = offset.I[1] + full_row_index
                        return (true, (mi, ri, tower[num_rows_removed+1:rock_level, :]), r, num_rows_removed)
                    elseif r == max_num_rocks
                        return (false, (mi, ri, tower[1:rock_level, :]), r, 0)
                    end

                    offset = CartesianIndex(rock_level + 3, 2)
                    if rock_level + 7 > size(tower, 1)
                        tower = [tower; zero(tower)]
                    end
                end
            end
        end
    end

    cache = Dict{Tuple{Int64,Int64,Matrix{Int64}},Tuple{Tuple{Int64,Int64,Matrix{Int64}},Int64,Int64}}()

    state = (1, 1, zeros(Int, 0, 7))
    total_rock_level = 0
    while num_rocks > 0
        if state in keys(cache)
            cycle_state, cycle_r, cycle_rl = _, state_r, state_rl = cache[state]
            while cycle_state != state
                cycle_state, r, rl = cache[cycle_state]
                cycle_r += r
                cycle_rl += rl
            end
            num_cycles, num_rocks = divrem(num_rocks, cycle_r)
            total_rock_level += num_cycles * cycle_rl
            if num_rocks > state_r
                num_rocks -= state_r
                total_rock_level += state_rl
                state = cache[state][1]
                continue
            end
        end
        full_row, new_state, r, rl = fall_until_full_row(state, num_rocks)
        if full_row
            cache[state] = (new_state, r, rl)
        end
        state = new_state
        num_rocks -= r
        total_rock_level += rl
    end
    total_rock_level + size(state[3], 1)
end

falling_rocks(data_17, 2022), falling_rocks(data_17, 10^12)

(3144, 1565242165201)

## [Day 18: Boiling Boulders](https://adventofcode.com/2022/day/18)

In [27]:
using StaticArrays

In [28]:
data_18 = readlines("18.txt")
cubes = [SVector(parse.(Int, xyz)...) for xyz in split.(data_18, ',')]

function surface_area(cubes)
    n = length(cubes)
    6 * n - 2 * sum(sum(abs.(cubes[i] - cubes[j])) == 1 for i in 1:n for j in i+1:n)
end

function exterior_surface_area(cubes)
    directions = [SVector(ntuple(x -> s * (x == i), 3)) for i in 1:3 for s in (-1, 1)]
    lo, hi = reduce((x, y) -> min.(x, y), cubes) .- 1, reduce((x, y) -> max.(x, y), cubes) .+ 1

    cubes = Set(cubes)
    unvisited = [lo]
    visited = Set{SVector{3,Int64}}()
    area = 0
    while !isempty(unvisited)
        (c = pop!(unvisited)) in visited && continue
        for d in directions
            n = c + d
            if all(lo .<= n .<= hi) && !(n in visited)
                if n in cubes
                    area += 1
                else
                    push!(unvisited, n)
                end
            end
        end
        push!(visited, c)
    end
    area
end

surface_area(cubes), exterior_surface_area(cubes)

(3650, 2118)

## [Day 19: Not Enough Minerals](https://adventofcode.com/2022/day/19)

In [29]:
data_19 = readlines("19.txt")
blueprints = [parse.(Int, first.(collect(eachmatch(r"(\d+)", bp)))) for bp in data_19]

function blueprint_quality(blueprint, time_horizon = 24)
    i, oo, co, bo, bc, go, gb = blueprint
    maxo, maxc, maxb = max(oo, co, bo, go), bc, gb
    truncate_state(h, (no, nc, nb, o, c, b)) =
        min.(
            (no, nc, nb, o, c, b),
            (maxo, maxc, maxb, maxo + (maxo - no) * h, maxc + (maxc - nc) * h, maxb + (maxb - nb) * h),
        )

    reward_to_come = [Dict((1, 0, 0, 0, 0, 0) => 0)]
    for t in 1:time_horizon
        r_t = Dict{NTuple{6,Int64},Int64}()
        for (s, r) in reward_to_come[end]
            no, nc, nb, o, c, b = s
            s_n = truncate_state(time_horizon - t - 1, (no, nc, nb, o + no, c + nc, b + nb))
            r_t[s_n] = max(get(r_t, s_n, 0), r)
            if o >= oo
                s_n = truncate_state(time_horizon - t - 1, (no + 1, nc, nb, o + no - oo, c + nc, b + nb))
                r_t[s_n] = max(get(r_t, s_n, 0), r)
            end
            if o >= co
                s_n = truncate_state(time_horizon - t - 1, (no, nc + 1, nb, o + no - co, c + nc, b + nb))
                r_t[s_n] = max(get(r_t, s_n, 0), r)
            end
            if o >= bo && c >= bc
                s_n = truncate_state(time_horizon - t - 1, (no, nc, nb + 1, o + no - bo, c + nc - bc, b + nb))
                r_t[s_n] = max(get(r_t, s_n, 0), r)
            end
            if o >= go && b >= gb
                s_n = truncate_state(time_horizon - t - 1, (no, nc, nb, o + no - go, c + nc, b + nb - gb))
                r_t[s_n] = max(get(r_t, s_n, 0), time_horizon - t + r)
            end
        end
        push!(reward_to_come, r_t)
    end
    maximum(values(reward_to_come[end]))
end

sum(prod.(enumerate(blueprint_quality.(blueprints)))), prod(blueprint_quality.(blueprints[1:3], 32))

(1081, 2415)

## [Day 20: Grove Positioning System](https://adventofcode.com/2022/day/20)

In [30]:
data_20 = readlines("20.txt")
sequence = parse.(Int, data_20)
numbered_sequence = collect(enumerate(sequence))

function mix!(sequence)
    for i in 1:length(sequence)
        j = findfirst(z -> first(z) == i, sequence)
        x = last(popat!(sequence, j))
        insert!(sequence, mod(j + x, 1:length(sequence)), (i, x))
    end
    sequence
end

function decrypt(sequence)
    j = findfirst(z -> last(z) == 0, sequence)
    sum(last(sequence[mod(j + i, 1:length(sequence))]) for i in (1000, 2000, 3000))
end

decrypt(mix!(numbered_sequence)),
decrypt(foldl((x, _) -> mix!(x), 1:10; init = collect(enumerate(sequence .* 811589153))))

(11616, 9937909178485)

## [Day 21: Monkey Math](https://adventofcode.com/2022/day/21)

In [31]:
using SymPy

In [32]:
data_21 = readlines("21.txt")

function parse_monkey_line(line)
    name, job = split(line, ": ")
    job = split(job)
    length(job) == 1 && return name => ((), () -> parse(Int, only(job)))
    a, op, b = job
    name => ((a, b), getfield(Main, Symbol(op)))
end

monkeys = Dict(parse_monkey_line.(data_21))

function monkey_yells(monkeys, match = false)
    if match
        monkeys = deepcopy(monkeys)
        monkeys["humn"] = ((), () -> sympy.Symbol("x"))
        monkeys["root"] = (monkeys["root"][1], -)
        yells = Dict{String,Any}()
    else
        yells = Dict{String,Int}()
    end

    while length(yells) < length(monkeys)
        for (m, (deps, op)) in monkeys
            if !(m in keys(yells)) && issubset(deps, keys(yells))
                yells[m] = op((d -> yells[d]).(deps)...)
            end
        end
    end
    yells
end

monkey_yells(monkeys)["root"], solve(monkey_yells(monkeys, true)["root"], sympy.Symbol("x"))

(51928383302238, Sym[3305669217840.00])

## [Day 22: Monkey Map](https://adventofcode.com/2022/day/22)

In [33]:
data_22 = readlines("22.txt")
grove_data = data_22[1:end-2]
grove = reduce(vcat, permutedims.(collect.(rpad.(grove_data, maximum(length.(grove_data))))))
movement = data_22[end]

rotate_ccw((x, y)) = (-y, x)
rotate_ccw(i::CartesianIndex) = CartesianIndex(rotate_ccw(i.I))
rotate_cw((x, y)) = (y, -x)
rotate_cw(i::CartesianIndex) = CartesianIndex(rotate_cw(i.I))
direction(i::CartesianIndex) = CartesianIndex(sign.(i.I))
direction((a, b)) = direction(b - a)
edge_indices((a, b)) = (a + i * direction(b - a) for i in 0:sum(abs.((b - a).I)))

function traverse_grove_flat(grove, movement)
    row_ranges = [findfirst(!=(' '), r):findlast(!=(' '), r) for r in eachrow(grove)]
    col_ranges = [findfirst(!=(' '), c):findlast(!=(' '), c) for c in eachcol(grove)]

    curr = findfirst(==('.'), grove[1:1, :])
    d = CartesianIndex(0, 1)
    m = 1
    while true
        n = findnext(in(('L', 'R')), movement, m)
        s, t = isnothing(n) ? (parse(Int, movement[m:end]), 'S') : (parse(Int, movement[m:n-1]), movement[n])
        for i in 1:s
            next = CartesianIndex(mod.((curr + d).I, (col_ranges[curr[2]], row_ranges[curr[1]]))...)
            grove[next] == '#' && break
            curr = next
        end
        t == 'S' && break
        d = t == 'L' ? rotate_ccw(d) : rotate_cw(d)
        m = n + 1
    end
    1000 * curr[1] + 4 * curr[2] + (d[1] == 0 ? 1 - d[2] : 2 - d[1])
end

function traverse_grove_cube(grove, movement)
    cube_size = Int(sqrt(count(==(' '), grove) / 6))
    start = findfirst(==('.'), grove[1:1, :])
    edges = [(start, start + CartesianIndex(cube_size - 1, 0))]
    vertex_neighborhoods = [Set((start,))]
    warp = Dict{NTuple{2,CartesianIndex{2}},NTuple{2,CartesianIndex{2}}}()

    for _ in 2:14
        a, b = last(edges)
        u = direction(b - a)
        if !(b + u in CartesianIndices(grove)) || grove[b+u] == ' '
            push!(edges, (b, b + rotate_ccw(b - a)))
            push!(vertex_neighborhoods, Set((b,)))
        elseif b + u + rotate_cw(u) in CartesianIndices(grove) && grove[b+u+rotate_cw(u)] != ' '
            push!(edges, (b + u + rotate_cw(u), b + u + rotate_cw(u) + rotate_cw(b - a)))
            push!(vertex_neighborhoods, Set((b, b + u, b + u + rotate_cw(u))))
        else
            push!(edges, (b + u, b + u + (b - a)))
            push!(vertex_neighborhoods, Set((b, b + u)))
        end
    end

    for _ in 1:7
        v = findfirst(length(n) == 3 for n in vertex_neighborhoods)
        matched_edge_pair = Tuple{eltype(edges),eltype(edges)}[]
        for x in vertex_neighborhoods[v]
            for (ei, e) in enumerate(edges)
                if x == e[1]
                    push!(matched_edge_pair, (e, e .+ Ref(rotate_cw(direction(e)))))
                elseif x == e[2]
                    push!(matched_edge_pair, (reverse(e), reverse(e) .+ Ref(rotate_cw(direction(e)))))
                else
                    continue
                end
                deleteat!(edges, ei)
            end
        end
        (i1, o1), (i2, o2) = matched_edge_pair
        for (x, y) in zip(zip(edge_indices(i1), edge_indices(o1)), edge_indices(i2))
            warp[x] = (y, first(i2) - first(o2))
        end
        for (x, y) in zip(zip(edge_indices(i2), edge_indices(o2)), edge_indices(i1))
            warp[x] = (y, first(i1) - first(o1))
        end
        filter!(Base.Fix1(isdisjoint, vertex_neighborhoods[v]), vertex_neighborhoods)
        for n in vertex_neighborhoods
            last(i1) in n && push!(n, last(i2))
            last(i2) in n && push!(n, last(i1))
        end
    end

    curr = start
    d = CartesianIndex(0, 1)
    m = 1
    while true
        n = findnext(in(('L', 'R')), movement, m)
        s, t = isnothing(n) ? (parse(Int, movement[m:end]), 'S') : (parse(Int, movement[m:n-1]), movement[n])
        for i in 1:s
            next, next_d = curr + d, d
            if !(next in CartesianIndices(grove)) || grove[next] == ' '
                next, next_d = warp[(curr, next)]
            end
            grove[next] == '#' && break
            curr, d = next, next_d
        end
        t == 'S' && break
        d = t == 'L' ? rotate_ccw(d) : rotate_cw(d)
        m = n + 1
    end
    1000 * curr[1] + 4 * curr[2] + (d[1] == 0 ? 1 - d[2] : 2 - d[1])
end

traverse_grove_flat(grove, movement), traverse_grove_cube(grove, movement)

(117054, 162096)

## [Day 23: Unstable Diffusion](https://adventofcode.com/2022/day/23)

In [34]:
data_23 = readlines("23.txt")
elf_grid = reduce(vcat, permutedims.(collect.(data_23)))

function diffuse_elves(elf_grid, t_max = 10)
    NSWE = CartesianIndices.(((-1:-1, -1:1), (1:1, -1:1), (-1:1, -1:-1), (-1:1, 1:1)))
    neighbors = union(NSWE...)
    elf_set = Set(findall(==('#'), elf_grid))

    for i in 1:t_max
        next_elf_set = filter(e -> !any(f in elf_set for f in Ref(e) .+ neighbors), elf_set)
        length(next_elf_set) == length(elf_set) && return elf_set, i
        from2to = Dict{CartesianIndex{2},CartesianIndex{2}}()
        for d in mod1.(i .- (1:4), 4)
            for e in elf_set
                if !(e in next_elf_set) && !any(f in elf_set for f in e .+ NSWE[d])
                    from2to[e] = e + NSWE[d][2]
                end
            end
        end
        to2from = Dict{CartesianIndex{2},CartesianIndex{2}}()
        for (f, t) in from2to
            if t in keys(to2from)
                delete!(to2from, t)
            else
                to2from[t] = f
            end
        end
        elf_set = union!(next_elf_set, keys(to2from), setdiff!(elf_set, values(to2from)))
    end
    elf_set, t_max
end

count_empty_ground_tiles(elf_set) = length(range(extrema(elf_set)...)) - length(elf_set)

count_empty_ground_tiles(diffuse_elves(elf_grid)[1]), diffuse_elves(elf_grid, typemax(Int))[2]

(3871, 925)

## [Day 24: Blizzard Basin](https://adventofcode.com/2022/day/24)

In [35]:
using DataStructures
using StaticArrays

In [36]:
data_24 = readlines("24.txt")
maze = reduce(vcat, permutedims.(collect.(data_24)))[2:end-1, 2:end-1]

function traverse_maze(maze, t0 = 0, swap = false)
    ls, rs, us, ds = (SVector.(Tuple.(findall(==(c), maze))) for c in ('<', '>', '^', 'v'))
    start = SVector(0, 1)
    goal = SVector(size(maze) .+ (0, 1))
    if swap
        start, goal = goal, start
    end

    obstacle_cache = Dict{Int64,Set{SVector{2,Int64}}}()
    function obstacles(t)
        get!(obstacle_cache, t) do
            Set(
                mod1.(x, size(maze)) for
                x in [ls .+ (SVector(0, -t),); rs .+ (SVector(0, t),); us .+ (SVector(-t, 0),); ds .+ (SVector(t, 0),)]
            )
        end
    end

    pq = PriorityQueue((start, t0) => t0)
    while first(first(peek(pq))) != goal
        (s, t), _ = dequeue_pair!(pq)
        for n in (s,) .+ (SVector(1, 0), SVector(-1, 0), SVector(0, 1), SVector(0, -1), SVector(0, 0))
            if (CartesianIndex(n...) in CartesianIndices(maze) || n == goal || n == start) && !(n in obstacles(t + 1))
                pq[(n, t + 1)] = t + 1
            end
        end
    end
    last(peek(pq))
end

traverse_maze(maze), traverse_maze(maze, traverse_maze(maze, traverse_maze(maze), true))

(221, 739)

## [Day 25: Full of Hot Air](https://adventofcode.com/2022/day/25)

In [37]:
data_25 = readlines("25.txt")

function snafu_sum(lines)
    snafu2int = Dict('2' => 2, '1' => 1, '0' => 0, '-' => -1, '=' => -2)
    int2snafu = Dict(v => k for (k, v) in snafu2int)
    total = sum([evalpoly(5, [snafu2int[y] for y in x]) for x in reverse.(collect.(data_25))])
    total_digits = pushfirst!(reverse(digits(total; base = 5)), 0)
    while any(>(2), total_digits)
        for i in 1:length(total_digits)
            if total_digits[i] > 2
                total_digits[i-1] += 1
                total_digits[i] -= 5
            end
        end
    end
    first(total_digits) == 0 && popfirst!(total_digits)
    prod([int2snafu[d] for d in total_digits])
end

snafu_sum(data_25)

"2-20=01--0=0=0=2-120"